In [ ]:
import os
from pathlib import Path
import shutil

def clean_name(name):
    """清理名称中的注释和特殊字符"""
    # 移除注释(#或:后的内容)
    for comment_char in ['#', ':']:
        if comment_char in name:
            name = name.split(comment_char)[0].rstrip()
    
    # 处理文件夹标记
    if name.rstrip().endswith(('/','\\')): 
        name = name.rstrip('/\\')
    elif '/ ' in name:
        name = name.split('/ ')[0]
    elif '\\ ' in name:
        name = name.split('\\ ')[0]
    
    # 移除前缀破折号和多余空格
    return name.strip('- ').strip()

def create_project_structure(structure_file):
    with open(structure_file, 'r', encoding='utf-8') as f:
        lines = [line.rstrip() for line in f.readlines() if line.strip()]
    
    # 获取项目根目录 (特殊处理第一行)
    first_line = lines[0]
    if 'project_dir:' in first_line:
        root_dir = first_line.split('project_dir:')[1].strip()
    else:
        root_dir = first_line
    
    # 用于跟踪目录层级
    path_stack = []
    current_indent = -1
    
    for line in lines[1:]:
        # 计算实际缩进
        indent = (len(line) - len(line.lstrip())) // 2
        
        # 清理并获取目录/文件名
        clean_line = clean_name(line.lstrip())
        if not clean_line:
            continue
            
        # 处理目录层级
        while path_stack and current_indent >= indent:
            path_stack.pop()
            current_indent -= 1
            
        # 更新当前路径
        path_stack.append(clean_line)
        current_indent = indent
        
            
            # 构建完整路径
        full_path = os.path.join(root_dir, *path_stack)
        
        try:
            # 创建目录或文件
            if '.' in clean_line and not clean_line.endswith(('/', '\\')):  # 文件
                # 确保父目录存在
                os.makedirs(os.path.dirname(full_path), exist_ok=True)
                
                # 检查文件是否存在
                if os.path.exists(full_path):
                    if os.path.getsize(full_path) > 0:
                        # 创建备份
                        file_path, file_ext = os.path.splitext(full_path)
                        backup_path = f"{file_path}_old{file_ext}~"
                        if os.path.exists(backup_path):
                            os.remove(backup_path)
                        shutil.move(full_path, backup_path)
                        print(f'备份文件: {full_path} -> {backup_path}')
                    else:
                        os.remove(full_path)
                
                # 创建新文件
                Path(full_path).touch()
                print(f'创建文件: {full_path}')
            else:  # 目录
                os.makedirs(full_path, exist_ok=True)
                print(f'创建目录: {full_path}')
                
        except Exception as e:
            print(f'处理 {full_path} 时出错: {str(e)}')
        
        prev_indent = indent        

def main():
    create_project_structure('./项目文件结构.txt')
    print("项目结构创建完成！")

if __name__ == '__main__':
    main()